# Forecast Analysis & Experimentation

This notebook shows how to work with forecasts interactively.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
from src.data import Fetcher, prepare_for_prophet
from src.models import ForecastModel
from src.analysis import ForecastAnalyzer

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Prepare Data

In [ ]:
# Fetch and prepare data
fetcher = Fetcher()
symbol = 'AAPL'

data = fetcher.fetch(symbol, '2023-01-01', '2024-10-31')
prophet_data = prepare_for_prophet(data)

print(f"Data prepared: {len(prophet_data)} records")

## 2. Train Model & Generate Forecast

In [ ]:
# Train model
model = ForecastModel()
model.train(prophet_data)

# Generate 90-day forecast
forecast = model.predict(periods=90)
print(f"Forecast generated: {len(forecast)} days")

## 3. Analyze Forecast

In [ ]:
# Initialize analyzer
analyzer = ForecastAnalyzer()

# Get future values
future_30 = analyzer.get_future_values(forecast, days=30)
future_90 = analyzer.get_future_values(forecast, days=90)

print(f"30-day forecast: {len(future_30)} records")
print(f"90-day forecast: {len(future_90)} records")

In [ ]:
# Find optimal sell date
optimal = analyzer.find_optimal_sell_date(forecast)

print("Optimal Sell Date:")
print(f"  Date: {optimal['date']}")
print(f"  Expected Price: ${optimal['price']:.2f}")
print(f"  Optimistic: ${optimal['price_optimistic']:.2f}")
print(f"  Pessimistic: ${optimal['price_pessimistic']:.2f}")
print(f"  Days from now: {optimal['days_from_now']}")

In [ ]:
# Calculate volatility
volatility = analyzer.calculate_volatility(forecast, window=90)

print("Volatility Metrics:")
print(f"  Std Dev: ${volatility['std_dev']:.2f}")
print(f"  Coefficient of Variation: {volatility['coefficient_of_variation']:.4f}")
print(f"  Avg Confidence Range: ${volatility['avg_confidence_range']:.2f}")

## 4. Visualize Forecast

In [ ]:
# Plot forecast
fig, ax = plt.subplots(figsize=(14, 7))

# Historical data
ax.plot(prophet_data['ds'], prophet_data['y'], 
        'k.', markersize=3, alpha=0.5, label='Historical')

# Forecast
future = analyzer.get_future_values(forecast, days=90)
ax.plot(future['ds'], future['yhat'], 'b-', linewidth=2, label='Forecast')

# Confidence interval
ax.fill_between(future['ds'], 
                future['yhat_lower'], 
                future['yhat_upper'],
                alpha=0.2, color='blue', label='95% CI')

ax.set_title(f'{symbol} - 90 Day Forecast', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price (USD)', fontsize=12)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Compare Different Forecast Periods

In [ ]:
# Generate forecasts for different periods
forecasts = {
    '30 days': model.predict(periods=30),
    '90 days': model.predict(periods=90),
    '180 days': model.predict(periods=180),
    '365 days': model.predict(periods=365)
}

# Plot all
fig, ax = plt.subplots(figsize=(14, 7))

colors = ['blue', 'green', 'red', 'purple']
for (label, fc), color in zip(forecasts.items(), colors):
    future = analyzer.get_future_values(fc, days=-1)
    ax.plot(future['ds'], future['yhat'], 
            label=label, linewidth=2, color=color)

ax.set_title('Forecast Comparison - Different Time Horizons', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price (USD)', fontsize=12)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Experiment: What-If Scenarios

In [ ]:
# Generate scenarios for different dates
dates_to_check = [
    forecast['ds'].max() - pd.Timedelta(days=60),
    forecast['ds'].max() - pd.Timedelta(days=30),
    forecast['ds'].max()
]

print("Price Scenarios:\n")
for date in dates_to_check:
    date_str = date.strftime('%Y-%m-%d')
    scenarios = analyzer.generate_scenarios(forecast, target_date=date_str)
    
    print(f"Date: {date_str}")
    print(f"  Optimistic: ${scenarios['optimistic']['price']:.2f}")
    print(f"  Expected:   ${scenarios['expected']['price']:.2f}")
    print(f"  Pessimistic: ${scenarios['pessimistic']['price']:.2f}")
    print()

## Next Steps

Try experimenting with:
- Different stocks
- Different time periods
- Different model parameters in `config.yaml`